## Stochastic ODE
$\frac{dy(t)}{dt} = -ky$ where decay rate coefficient $k$ is considered to be a random variable $k(\theta)$ with pdf $f(k)$ and mean value $\bar{k}$.

In [5]:
#Import necessary modules
import numpy as np
from scipy.sparse import diags, csr_matrix, kron
from scipy.linalg import solve_sylvester, schur, expm
from scipy.sparse.linalg import spsolve, cg
from scipy.integrate import quad
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import time
import random
%config InlineBackend.figure_format = 'svg'

In [6]:
def plot_sol(X,Y,U,i):
    plt.figure(i)
    xline = np.reshape(X, -1)
    yline = np.reshape(Y, -1)
    zline = np.reshape(U, -1)
    plt.imshow(U, extent=[-1,1,-1,1], origin='lower')
    plt.colorbar()
    plt.axis(aspect='image')
    plt.xlabel('x')
    plt.ylabel('y')


def compute_err(U, U_exact):
    n = len(U)
    err_inf = 0
    err_sq = 0
    for i in range(0,n):
        for j in range(0,n):
            err_sq += np.absolute(U_exact[i][j] - U[i][j])**2
            if np.absolute(U_exact[i][j] - U[i][j]) > err_inf:
                err_inf = np.absolute(U_exact[i][j] - U[i][j])
        
    err_sq = (err_sq * h**2)**0.5
    return err_inf, err_sq


In [15]:
import numpy as np
from scipy.sparse import diags, kron
from scipy.sparse.linalg import spsolve, cg
from scipy.special import hermite
from scipy.integrate import quad
from sympy import *
from scipy import sparse

def inner_product(i,j,l,squared=False):
    if squared == True:
        I = lambda xi: hermite(i,xi)*hermite(j,xi)
    else:
        I = lambda xi: hermite(i,xi)*hermite(j,xi)*hermite(l,xi)
    return quad(I,0,1)[0]

def f(l,y,p):
    sum = 0
    for i in range(0,p+1):
        for j in range(0,p+1):
            sum += inner_product(i,j,l)*y
    return (-1/inner_product(l,l,l,True))*sum

p = 3 #polynomial parameter
y = 1 #initial condition
t = 0.1 #time step

for l in range(0,p+1):
    k1 = t*f(l,y,p)
    k2 = t*f(l,y+(k1/2),p)
    y = y + k2


